[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/UNSW-COMP9414/Tutorials/blob/main/Week08/COMP9414-Week08-Tree-Models-Solution.ipynb)

# Training and Assessing Tree-based Models with Scikit-Learn

**COMP9414 W08 Tutorial**

- Instructor: Gustavo Batista
- School of Computer Science and Engineering, UNSW Sydney
- Notebook designed by Gustavo Batista
- Last Update 24th October 2024

In this week's tutorial, we will train and assess three tree-based models: decision trees, bagging of trees and random forests. We will start with the Iris dataset to gain intuition about cutpoints and splitting criteria. Later, we will use the Wine Quality dataset to train, evaluate and compare our tree-based model.

We will use the Scikit-Learn library, which provides an extensive collection of Machine Learning classifiers and regressors, among other useful functions for data preparation and model assessment.

## Technical prerequisites

You will need the following packages installed to run this notebook:

1. Numpy
2. Pandas
3. Matplotlib
4. Seaborn
5. Scikit-learn

These libraries are often found in most installations. If they are not installed on your system, you can install them using the `pip` or `conda` commands.

In [ ]:
# NumPy and matplotlib libraries for numerical computation and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn libraries for dataset
from sklearn.datasets import load_iris

# Scikit-learn libraries for models
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestClassifier, RandomForestRegressor

# Scikit-learn libraries for model assessment and hyperparameter search
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score

## The Iris dataset

We will start with a simple decision tree classifier using the Iris dataset. 

The Iris dataset is a classic benchmark dataset. It contains 150 samples of iris flowers, divided equally among three species: setosa, versicolor, and virginica. Each sample has four numerical features: sepal length, sepal width, petal length, and petal width, all measured in centimetres. The target variable indicates the flower's species, encoded as 0 (Setosa), 1 (Versicolor), and 2 (Virginica).

Like MNIST, this dataset is very popular for benchmarking Machine Learning models. It is even available as part of the Scikit-learn installation.

We can use the following command to load the dataset.

In [ ]:
# Load the Iris dataset
iris = load_iris()

# Convert it into a Pandas DataFrame
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Display the first few rows of the dataset
print(iris_df.head())

## Exploratory data analysis

Before building our classifiers, it is good to get a better understanding of the data. A extremelly popular plot is known as the scatterplot matrix (among other names). This is matrix of scatterplots comparing pairs of attributes.

We can easily create a scatterplot matrix using the `pairplot` function of the Seaboarn library. The next cell creates this plot for the Iris dataset. Before creating the plot, we will rename the classes, from 0, 1 and 2 to the actual class labels: setosa, versicolor and virginica.

In [ ]:
# Map target integers to class names for better visualization
iris_df['species'] = iris_df['target'].map({0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'})

# Create a scatter plot matrix
sns.pairplot(iris_df, hue='species', diag_kind='kde', palette='bright', markers=["o", "s", "D"])

# Display the plot
plt.show()

The scatterplot matrix shows that the class setosa can be easily separated from the other two. For instance, simple cuts such as petal length around two and petal width around one will create a terminal node for all setosa cases.

In the next section, we will use this information to assess the node splitting measures Gini index and cross-entropy.

## Calculation Gini and Cross-entropy

Let's start with an exercise to compute the Gini index and cross-entropy. We can calculate the Gini index with the following equation:

$$G = \sum_{k=1}^{K} \hat{p}_{mk}(1-\hat{p}_{mk}) $$

While the cross-entropy is defined as:

$$D = - \sum_{k=1}^{K} \hat{p}_{mk} \log_2 \hat{p}_{mk} $$

Where $K$ is the number of classes and $\hat{p}_{mk}$ is the class distribution for the class $k$ in a region $m$.

### Exercise

Implement the functions ``compute_gini(p)`` and ``compute_cross_entropy(p)`` that compute the Gini index and cross-entropy for a vector of class distributions ``p``. The vector ``p`` has $K$ values in the $[0, 1]$ range that sum up to one.

In [ ]:
def compute_cross_entropy(p):
    """
    Compute the cross-entropy of a probability distribution.

    Parameters:
    - p (numpy.ndarray): 1D array of class probabilities (must sum to 1).

    Returns:
    - float: The cross-entropy value.
    """
    ... # TODO
    
def compute_gini(p):
    """
    Compute the Gini impurity of a probability distribution.

    Parameters:
    - p (numpy.ndarray): 1D array of class probabilities (must sum to 1).

    Returns:
    - float: The Gini impurity value.
    """
    ... # TODO

In [ ]:
# Answer

def compute_cross_entropy(p):
    """
    Compute the cross-entropy of a probability distribution.

    Parameters:
    - p (numpy.ndarray): 1D array of class probabilities (must sum to 1).

    Returns:
    - float: The cross-entropy value.
    """
    return -np.sum(p * np.log2(p + 1e-15))  # Add small value to avoid log(0)

def compute_gini(p):
    """
    Compute the Gini impurity of a probability distribution.

    Parameters:
    - p (numpy.ndarray): 1D array of class probabilities (must sum to 1).

    Returns:
    - float: The Gini impurity value.
    """
    return np.sum(p * (1 - p))

In [ ]:
# Test cases
# These are some test cases to help you check if your implementation is correct. Each test case has an associated expected response.

print(compute_cross_entropy(np.array([0.5, 0.5])))   # 1
print(compute_cross_entropy(np.array([0, 1])))       # 0, if you got a NaN, it is being caused by a log(0)
print(compute_gini(np.array([0.5, 0.5])))            # 0.5
print(compute_gini(np.array([0, 1])))                # 0

## Exercise

We will also need a function that computes the class distribution given an array of class labels. For instance:

1. [0, 0, 0, 1, 1, 1] - returns a class distribution [0.5, 0.5].
2. [0, 1, 1, 1, 1, 1] - returns a class distribution [1/6, 5/6].

Try to implement a generic function that works for any number of class labels of any data type such as natural numbers 0, 1, 2, ... and strings 'a', 'b', 'c', etc.

Implement a function ``compute_class_distribution(labels)`` that returns a vector with the class distribution of the class labels in ``labels``.

In [ ]:
def compute_class_distribution(labels):
    """
    Compute the proportion of each class in the given labels.

    Parameters:
    - labels (numpy.ndarray): 1D array of class labels.

    Returns:
    - numpy.ndarray: 1D array of class proportions.
    """
    ... # TODO

In [ ]:
# Answer

def compute_class_distribution(labels):
    """
    Compute the proportion of each class in the given labels.

    Parameters:
    - labels (numpy.ndarray): 1D array of class labels.

    Returns:
    - numpy.ndarray: 1D array of class proportions.
    """
    total = len(labels)
    return np.array([np.sum(labels == i) / total for i in np.unique(labels)])

In [ ]:
# Test cases
# These are some test cases to help you check if your implementation is correct. Each test case has an associated expected response.

print(compute_class_distribution(np.array([0, 0, 0, 1, 1, 1])))               # [0.5, 0.5]
print(compute_class_distribution(np.array([0, 0, 0, 0, 0, 1])))               # [5/6, 1/6]
print(compute_class_distribution(np.array(['a', 'a', 'b', 'b', 'c', 'c'])))   # [1/3, 1/3, 1/3]

### Exercise

Finally, let's compute the cut points for the Gini index and cross-entropy for a given numeric attribute $X$.

The attribute will assume a series of values for different examples, for instance:

$X$ = [6, 2, 8, 4, 6, 9, 1, 3, 5, 7, 9]

We will sort the unique values of $X$:

$X$ = [1, 2, 3, 4, 5, 6, 7, 8, 9]

And we will compute the mid-points between consecutive values:

cut-points = [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5]

Implement a function ``compute_cut_points(X)`` that returns an array of cut-points given an input array of values for a feature ``X``.

In [ ]:
def compute_cut_points(X):
    """
    Compute the cut-points as mid-points for consecutive values of X.

    Parameters:
    - X (numpy.ndarray): 1D array of feature values.

    Returns:
    - numpy.ndarray: 1D array of cut-points.
    """
    ... # TODO

In [ ]:
def compute_cut_points(X):
    """
    Compute the cut-points as mid-points for consecutive values of X.

    Parameters:
    - X (numpy.ndarray): 1D array of feature values.

    Returns:
    - numpy.ndarray: 1D array of cut-points.
    """
    sorted_X = np.sort(np.unique(X))
    cutpoints = (sorted_X[:-1] + sorted_X[1:]) / 2
    return cutpoints

In [ ]:
# Test cases
# These are some test cases to help you check if your implementation is correct. Each test case has an associated expected response.

print(compute_cut_points(np.array([6, 2, 8, 4, 6, 9, 1, 3, 5, 7, 9])))      # [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5]

## Putting everything together

Let's use the implemented functions to measure the Gini index and cross-entropy for the petal length attribute.

### Exercise

Implement a code that measures the Gini index and cross-entropy for all cut-points of the petal length attribute. We have done most of the code for you. You just need to fill in a few gaps with calls to your functions.

In [ ]:
# Extract petal length and width features and the target variable
petal_length = iris_df['petal length (cm)'].values
petal_width = iris_df['petal width (cm)'].values
target = iris_df['target'].values

# Compute the cut-points for the petal_length attribute
cutpoints = ...                                                # TODO

# Store results for cross-entropy and Gini
results = []

# Iterate over each cutpoint
for cut in cutpoints:
    # Split the data into two groups based on the cutpoint
    left_group = target[petal_length < cut]
    right_group = target[petal_length >= cut]

    # Compute class distributions for both groups
    p_left = ...                                               # TODO
    p_right = ...                                              # TODO

    # Compute cross-entropy for both groups
    ce_left = ...                                              # TODO
    ce_right = ...                                             # TODO

    # Compute Gini impurity for both groups
    gini_left = ...                                            # TODO
    gini_right = ...                                           # TODO
    
    # Weighted averages of cross-entropy and Gini impurity
    total_size = len(left_group) + len(right_group)
    weighted_ce = (len(left_group) / total_size) * ce_left + (len(right_group) / total_size) * ce_right
    weighted_gini = (len(left_group) / total_size) * gini_left + (len(right_group) / total_size) * gini_right

    # Store the cutpoint and corresponding metrics
    results.append((cut, weighted_ce, weighted_gini))

# Convert results to DataFrame for easier plotting
metrics_df = pd.DataFrame(results, columns=['Cutpoint', 'Cross-Entropy', 'Gini'])

# Identify the cutpoint with minimal cross-entropy
min_ce_cutpoint = metrics_df.loc[metrics_df['Cross-Entropy'].idxmin(), 'Cutpoint']

# Plot Cross-Entropy and Gini Impurity for each Cutpoint
plt.figure(figsize=(12, 6))
plt.plot(metrics_df['Cutpoint'], metrics_df['Cross-Entropy'], marker='o', linestyle='-', label='Cross-Entropy', color='b')
plt.plot(metrics_df['Cutpoint'], metrics_df['Gini'], marker='s', linestyle='--', label='Gini Impurity', color='g')
plt.axvline(x=min_ce_cutpoint, color='r', linestyle='--', label=f'Min CE Cutpoint: {min_ce_cutpoint:.2f}')
plt.title('Cross-Entropy and Gini vs. Cutpoint for Petal Length')
plt.xlabel('Cutpoint (Petal Length)')
plt.ylabel('Metric Value')
plt.legend()
plt.show()

# Scatter plot of Petal Width vs. Petal Length with the minimal CE cutpoint
plt.figure(figsize=(12, 6))
plt.scatter(petal_length, petal_width, c=target, cmap='viridis', edgecolor='k', s=50)
plt.axvline(x=min_ce_cutpoint, color='r', linestyle='--', label=f'Min CE Cutpoint: {min_ce_cutpoint:.2f}')
plt.title('Petal Width vs. Petal Length with Minimal Cross-Entropy Cutpoint')
plt.xlabel('Petal Length (cm)')
plt.ylabel('Petal Width (cm)')
plt.legend()
plt.show()

In [ ]:
# Answer

# Extract petal length and width features and the target variable
petal_length = iris_df['petal length (cm)'].values
petal_width = iris_df['petal width (cm)'].values
target = iris_df['target'].values

# Compute the cut-points for the petal_length attribute
cutpoints = compute_cut_points(petal_length)

# Store results for cross-entropy and Gini
results = []

# Iterate over each cutpoint
for cut in cutpoints:
    # Split the data into two groups based on the cutpoint
    left_group = target[petal_length < cut]
    right_group = target[petal_length >= cut]

    # Compute class distributions for both groups
    p_left = compute_class_distribution(left_group)
    p_right = compute_class_distribution(right_group)

    # Compute cross-entropy for both groups
    ce_left = compute_cross_entropy(p_left)
    ce_right = compute_cross_entropy(p_right)

    # Compute Gini impurity for both groups
    gini_left = compute_gini(p_left)
    gini_right = compute_gini(p_right)
    
    # Weighted averages of cross-entropy and Gini impurity
    total_size = len(left_group) + len(right_group)
    weighted_ce = (len(left_group) / total_size) * ce_left + (len(right_group) / total_size) * ce_right
    weighted_gini = (len(left_group) / total_size) * gini_left + (len(right_group) / total_size) * gini_right

    # Store the cutpoint and corresponding metrics
    results.append((cut, weighted_ce, weighted_gini))

# Convert results to DataFrame for easier plotting
metrics_df = pd.DataFrame(results, columns=['Cutpoint', 'Cross-Entropy', 'Gini'])

# Identify the cutpoint with minimal cross-entropy
min_ce_cutpoint = metrics_df.loc[metrics_df['Cross-Entropy'].idxmin(), 'Cutpoint']

# Plot Cross-Entropy and Gini Impurity for each Cutpoint
plt.figure(figsize=(12, 6))
plt.plot(metrics_df['Cutpoint'], metrics_df['Cross-Entropy'], marker='o', linestyle='-', label='Cross-Entropy', color='b')
plt.plot(metrics_df['Cutpoint'], metrics_df['Gini'], marker='s', linestyle='--', label='Gini Impurity', color='g')
plt.axvline(x=min_ce_cutpoint, color='r', linestyle='--', label=f'Min CE Cutpoint: {min_ce_cutpoint:.2f}')
plt.title('Cross-Entropy and Gini vs. Cutpoint for Petal Length')
plt.xlabel('Cutpoint (Petal Length)')
plt.ylabel('Metric Value')
plt.legend()
plt.show()

# Scatter plot of Petal Width vs. Petal Length with the minimal CE cutpoint
plt.figure(figsize=(12, 6))
plt.scatter(petal_length, petal_width, c=target, cmap='viridis', edgecolor='k', s=50)
plt.axvline(x=min_ce_cutpoint, color='r', linestyle='--', label=f'Min CE Cutpoint: {min_ce_cutpoint:.2f}')
plt.title('Petal Width vs. Petal Length with Minimal Cross-Entropy Cutpoint')
plt.xlabel('Petal Length (cm)')
plt.ylabel('Petal Width (cm)')
plt.legend()
plt.show()

The plots show that the Gini index and cross-entropy are strongly correlated. Both indicate that the best cut-point is around 2.45, which gives us a good separation margin between the classes setosa and versicolor.

## Training our first decision tree classifier

Let's train our first decision tree classifier using the iris dataset. For now, we will only focus on training and model interpretability. In the next section, we will also evaluate and compare the decision tree models with their ensemble versions.

The next cell will split the Pandas dataframe into input features and class attribute and use the Scikit-learn library to create and a graphical representation of the tree.

In [ ]:
# Divide the DataFrame into attributes (features) and class labels
X = iris_df.iloc[:, :-2]  # All columns except the last one (features)
y = iris_df.iloc[:, -1]   # The last column (class labels)

# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the classifier on the training data
clf.fit(X, y)

# Plot the decision tree
plt.figure(figsize=(15, 10))
tree.plot_tree(clf, feature_names=list(X.columns), class_names=list(iris.target_names), filled=True)
plt.title("Decision Tree Trained on Iris Dataset")
plt.show()

### Description of the tree model

We can see how the decision tree splits the cases into regions. The root split is petal length <= 2.45, as we expected, creating a leaf note for the setosa class with all 50 cases of that plant. 

As we go down in the tree, we can see that some cases of Versicolor and Virginia are simple to separate, creating large leaf nodes with 47 (Versicolor) and 43 cases (Virginia) out of 50 cases of each class. However, other cases are difficult to separate, creating leaf nodes with one, two, or three cases. These nodes are prone to overfitting.

The next cell visualises all cut points and regions created for this decision tree. The cell is ready to run. You can change the attributes in the scatterplot to see how the decision tree splits across the other attributes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.datasets import load_iris

def plot_decision_tree_boundary(df, attribute1, attribute2):
    """
    Plots the decision boundaries for a Decision Tree Classifier using two selected attributes.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - attribute1 (str): The name of the first attribute to use for plotting.
    - attribute2 (str): The name of the second attribute to use for plotting.

    Returns:
    None
    """

    # Extract the selected attributes and class labels
    X = df[[attribute1, attribute2]].values
    y = df['target'].values

    # Initialize and train the Decision Tree Classifier
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X, y)

    # Define the mesh grid for decision boundaries
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

    # Predict the class for each point in the mesh grid
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the decision boundaries
    plt.figure(figsize=(10, 6))
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    plt.contourf(xx, yy, Z, cmap=cmap_light, alpha=0.6)

    # Plot the training points
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', s=50, cmap='viridis')
    plt.xlabel(attribute1)
    plt.ylabel(attribute2)
    plt.title(f'Decision Tree Decision Boundaries ({attribute1} vs. {attribute2})')
    plt.colorbar(scatter, label='Target Class')

    # Plot decision cutpoints (vertical or horizontal lines)
    n_nodes = clf.tree_.node_count
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold

    for i in range(n_nodes):
        if feature[i] == 0:  # Attribute 1 (x-axis)
            plt.axvline(x=threshold[i], color='r', linestyle='--', label=f'Cutpoint: {threshold[i]:.2f}')
        elif feature[i] == 1:  # Attribute 2 (y-axis)
            plt.axhline(y=threshold[i], color='b', linestyle='--', label=f'Cutpoint: {threshold[i]:.2f}')

    # Avoid duplicate legend entries
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='best')

    plt.grid(True)
    plt.show()

# Example usage: Plot using 'petal length (cm)' and 'petal width (cm)'
plot_decision_tree_boundary(iris_df, 'petal length (cm)', 'petal width (cm)')

Let's use a larger and more complicated dataset that will allow us to train and compare trees and forest models.

## The wine quality dataset.

The Wine Quality dataset contains the chemical properties of red and white wines and their quality ratings, scored on a scale from 0 to 10. Each row represents a specific wine sample, with features including fixed acidity, volatile acidity, citric acid, chlorides, and density.

We will use this dataset for regression (predicting exact quality scores) and classification (predicting if a wine is bad, for scores of 0 to 5, or good, for scores of 6 to 10).

Let's start by loading the dataset from the UCI data repository, separating the features from the target attribute and creating a binary class label where 0 corresponds to scores below and equal to 5 and 1 to scores greater than 5.

In [ ]:
# Load the Wine Quality dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine_df = pd.read_csv(url, delimiter=';')

# Separate features and target
X = wine_df.iloc[:, :-1]  # All columns except the last (features)
y = wine_df.iloc[:, -1]   # Last column (quality score)

# Convert wine quality into binary classes for classification (>=6 as good, <6 as bad)
y_class = (y >= 6).astype(int)

The wine dataset does not have a standard split of training and test partitions. So, we will create one with 80% for training and 20% for testing. We will call the Scikit-learn function `train_test_split` twice, as we have a classification and a regression target attribute.

As you will see, we will often use ``random_state=42``. This statement sets the seed of the random number generator. In most cases, we use this statement to enforce that different executions will provide the same results, even when the algorithms involve stochastic decisions. 

In the next cell, ``random_state=42`` plays an important role, as it enforces that the two executions of ``train_test_split`` will provide the same data partitioning. Notice that the first call splits input features and class labels, but the second only splits the regression targets and uses the input split of the first call.

In [ ]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)
_, _, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2, random_state=42)

## Training tree-based classifiers and regressors

Great, we can now train our models. We will show you how to do it for the decision tree classifier and let you do the same for the bagging and random forest classifiers. We will use the default hyperparameters for now and see how to search for them later.

In [ ]:
# Initialize the decision tree model for classification
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train_class)
y_pred_class = dt_classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize decision tree model for regression
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train, y_train_reg)
y_pred_reg = dt_regressor.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

Great, you should have got an accuracy of around 72% abd a MSE of 0.61. Remember that MSE is an error measure, so lower values are better. Accuracy is a measure of correctness, so higher values are better.

### Exercise

It is your turn. We will do the same for the bagging of decision trees and the random forest model.

We start by bagging decision trees. We have created an initial code; you just need to fill in the missing parts.

In [ ]:
# Initialize the bagging of trees model for classification
bagging_classifier = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)
... # TODO: fit the model to training data
... # TODO: predict the class of test instances
... # TODO: measure and print the accuracy of the model on test data

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize bagging of tree model for regression
bagging_regressor = BaggingRegressor(estimator=DecisionTreeRegressor(), random_state=42)
... # TODO: fit the model to training data
... # TODO: predict the target of the test instances
... # TODO: measure and print the MSE

In [ ]:
# Answer

# Initialize the bagging of trees model for classification
bagging_classifier = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)
bagging_classifier.fit(X_train, y_train_class)
y_pred_class = bagging_classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize bagging of tree model for regression
bagging_regressor = BaggingRegressor(estimator=DecisionTreeRegressor(), random_state=42)
bagging_regressor.fit(X_train, y_train_reg)
y_pred_reg = bagging_regressor.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

Here are a couple of observations about the bagging of decision trees

1. The technique significantly improves accuracy and reduced MSE. The reduction in MSE is notable.
2. Bagging is a technique that can be applied to different classifiers. Therefore, we need to specify a (base) estimator. You can use any base estimator besides decision trees, including neural nets.

Next, we will create a similar code for the Random Forest classifier. Although it is very similar to bagging of tree classifiers, the random forest only uses decision trees as base estimators.

In [ ]:
# Initialize the Random Forest model for classification
rf_classifier = RandomForestClassifier(random_state=42)
... # TODO: fit the model to training data
... # TODO: predict the class of test instances
... # TODO: measure and print the accuracy of the model on test data

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize bagging of tree model for regression
rf_regressor = RandomForestRegressor(random_state=42)
... # TODO: fit the model to training data
... # TODO: predict the target of the test instances
... # TODO: measure and print the MSE

In [ ]:
# Answer

# Initialize the bagging of trees model for classification
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train_class)
y_pred_class = rf_classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize bagging of tree model for regression
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train_reg)
y_pred_reg = rf_regressor.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

This time, we see minor improvements in accuracy and MSE compared to the bagging model.

## Searching for hyperparameters

Let's see if we can improve the performance of the classifiers and regressors for the Wine dataset. We will start with the decision tree model and show you how it is done, then let you code a similar search for the ensemble models.

We will use the Scikit-learn class GridSearchCV to implement the search procedure. GridSearchCV exhaustively searches through a predefined grid of hyperparameter combinations. The search performs k-fold cross-validation for each hyperparameter combination. In these experiments, we will perform 5-fold cross-validation.

For the tree models, we have the following recommendations:

1. ``max_depth``: Sets the maximum depth of the tree, controlling how deep the tree can grow. Deeper trees may overfit, while shallower trees generalize better.
2. ``min_samples_split``: The minimum number of samples required to split an internal node. Higher values prevent over-splitting and control overfitting by enforcing larger splits.
3. ``min_samples_leaf``: The minimum number of samples a leaf node must contain. Larger values create broader leaves.
3. ``ccp_alpha``: This is the cost complexity pruning parameter. A higher value increases pruning.

In [ ]:
# Define parameter grid
param_grid = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'ccp_alpha': [0.0, 0.01, 0.1]
}

# Initialize the Decision Tree model
dt_classifier = DecisionTreeClassifier(random_state=42)

# Perform Grid Search for Classification
grid_search_clf = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_clf.fit(X_train, y_train_class)

print("Best Classifier Parameters:", grid_search_clf.best_params_)
best_clf = grid_search_clf.best_estimator_
y_pred_class = best_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize the Regression Tree model
dt_regressor = DecisionTreeRegressor(random_state=42)

# Perform Grid Search for Regression
grid_search_reg = GridSearchCV(estimator=dt_regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_reg.fit(X_train, y_train_reg)

print("\nBest Regressor Parameters:", grid_search_reg.best_params_)
best_reg = grid_search_reg.best_estimator_
y_pred_reg = best_reg.predict(X_test)
print("\nRegression Performance:")
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

We can see some improvement in MSE, but it cannot achieve the level of the ensemble methods. Our search did not improve the classification performance, though.

## Exercise

Now it is your turn. Let's see if we can improve the performance of bagging decision trees with hyperparameter optimisation. Regarding the hyperparameters to search, we will search the same ones used for the trees and one specific of the ensemble: ``n_estimators``. This hyperparameter controls the number of trees created.

We have made most of the code and will let you complete a few gaps. 

In [ ]:
# Define parameter grid for bagging models
param_grid = {
    'estimator__max_depth': [3, 5, 10],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4],
    'estimator__ccp_alpha': [0.0, 0.01, 0.1],
    'n_estimators': [10, 50, 100]
}

# Initialize the Bagging classification model with Decision Trees as base estimators
bagging_clf = ...                                                  # TODO: initialise the decision tree bagging classifier

# Perform Grid Search for Bagging Classifier
grid_search_clf = ...                                              # TODO: create and initialise the grid search object
grid_search_clf.fit(X_train, y_train_class)

print("Best Classifier Parameters:", grid_search_clf.best_params_)
best_clf = grid_search_clf.best_estimator_
y_pred_class = best_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize the Bagging regression model with Decision Trees as base estimators
bagging_reg = ...                                                # TODO: initialise decision tree bagging regressor

# Perform Grid Search for Bagging Regressor
grid_search_reg = ...                                            # TODO: create and initialise the grid search object
grid_search_reg.fit(X_train, y_train_reg)

print("\nBest Regressor Parameters:", grid_search_reg.best_params_)
best_reg = grid_search_reg.best_estimator_
y_pred_reg = best_reg.predict(X_test)
print("\nRegression Performance:")
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

In [ ]:
# Answer

# Define parameter grid for bagging models
param_grid = {
    'estimator__max_depth': [3, 5, 10],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4],
    'estimator__ccp_alpha': [0.0, 0.01, 0.1],
    'n_estimators': [10, 50, 100]
}

# Initialize the Bagging classification model with Decision Trees as base estimators
bagging_clf = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42)

# Perform Grid Search for Bagging Classifier
grid_search_clf = GridSearchCV(estimator=bagging_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_clf.fit(X_train, y_train_class)

print("Best Classifier Parameters:", grid_search_clf.best_params_)
best_clf = grid_search_clf.best_estimator_
y_pred_class = best_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize the Bagging regression model with Decision Trees as base estimators
bagging_reg = BaggingRegressor(estimator=DecisionTreeRegressor(random_state=42), random_state=42)

# Perform Grid Search for Bagging Regressor
grid_search_reg = GridSearchCV(estimator=bagging_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_reg.fit(X_train, y_train_reg)

print("\nBest Regressor Parameters:", grid_search_reg.best_params_)
best_reg = grid_search_reg.best_estimator_
y_pred_reg = best_reg.predict(X_test)
print("\nRegression Performance:")
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

### Exercise

As our final task, we will implement a hyperparameter search for the Random Forest classifier and regressor. Again, we use the same hyperparameter grid for bagging trees.

We have implemented most of the code, leaving just a few parts for you to complete.

In [ ]:
# Define parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [3, 5, 10],         # Maximum depth of each tree
    'min_samples_split': [2, 5, 10], # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],   # Minimum samples required to form a leaf
    'ccp_alpha': [0.0, 0.01, 0.1]    # Complexity parameter for pruning
}

# Initialize the Random Forest model for classification
rf_classifier = ...                                                  # TODO: initialise the Random Forest classifier

# Perform Grid Search for Random Forest Classifier
grid_search_clf = ...                                                # TODO: create and initialise the grid search object
grid_search_clf.fit(X_train, y_train_class)

print("Best Classifier Parameters:", grid_search_clf.best_params_)
best_clf = grid_search_clf.best_estimator_
y_pred_class = best_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize the Random Forest model for regression
rf_regressor = ...                                                   # TODO: initialise the Random Forest regressor

# Perform Grid Search for Random Forest Regressor
grid_search_reg = ...                                                # TODO: create and initialise the grid search object
grid_search_reg.fit(X_train, y_train_reg)

print("\nBest Regressor Parameters:", grid_search_reg.best_params_)
best_reg = grid_search_reg.best_estimator_
y_pred_reg = best_reg.predict(X_test)
print("\nRegression Performance:")
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

In [ ]:
# Answer

# Define parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [3, 5, 10],         # Maximum depth of each tree
    'min_samples_split': [2, 5, 10], # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],   # Minimum samples required to form a leaf
    'ccp_alpha': [0.0, 0.01, 0.1]    # Complexity parameter for pruning
}

# Initialize the Random Forest model for classification
rf_classifier = RandomForestClassifier(random_state=42)

# Perform Grid Search for Random Forest Classifier
grid_search_clf = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_clf.fit(X_train, y_train_class)

print("Best Classifier Parameters:", grid_search_clf.best_params_)
best_clf = grid_search_clf.best_estimator_
y_pred_class = best_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Generate confusion matrix and plot it
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Initialize the Random Forest model for regression
rf_regressor = RandomForestRegressor(random_state=42)

# Perform Grid Search for Random Forest Regressor
grid_search_reg = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_reg.fit(X_train, y_train_reg)

print("\nBest Regressor Parameters:", grid_search_reg.best_params_)
best_reg = grid_search_reg.best_estimator_
y_pred_reg = best_reg.predict(X_test)
print("\nRegression Performance:")
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

# Conclusion

Today, we have learned how to implement and evaluate decision tree classifiers and regressors, including ensemble models. These models are excellent choices, particularly if the data is tabular.

Unfortunately, the Scikit-learn implementation of tree models is pretty basic. For instance, these models cannot handle qualitative (discrete) attributes. However, it is natural for tree models to create splits such as $X_1$ == 'red' and $X_1$ != 'green'.

Two more advanced implementations of ensemble decision trees are the highly popular [XGBoost](https://xgboost.readthedocs.io/) and the highly efficient [LightGBM](https://lightgbm.readthedocs.io/). These are variations of the [Random Forest classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) that use [boosting](https://en.wikipedia.org/wiki/Boosting_(machine_learning)) to implement the ensembles.

We hope that you have enjoyed the Machine Learning tutorials!